<img src="http://milepro.com/wp-content/uploads/2014/01/Travel-Credit-Cards-1024x606.jpg" style="width:200px; float: left; padding-right: 10px"/>
<h2 style="font-face: verdana; font-size: 32px;">Predict credit card customer churn<br>with IBM Watson Machine Learning</h2>
<h3 style="font-face: verdana; font-size: 16px;">Part 1: Data Exploration</h3>

## Churn Rate Trend

In [1]:
from io import StringIO
import requests
import json
import pandas as pd
import numpy as np

<div style="background-color: #e7f2fa; color: #333">
  <h3 style="background-color: #6ab0de">CHURN DATA</h3>
  <p>Load churn_data.csv from IBM Bluemix Object Store.</p>
</div>

In [2]:
import os, pandas as pd

churnDataRate = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/churn_data.csv',header=None)
churnDataRate.columns = ['YEAR', 'QUARTER', 'QUARTER_YEAR', 'CHURN_RATE']
churnDataRate.head()



,YEAR,QUARTER,QUARTER_YEAR,CHURN_RATE
0,2014,1,1Q14,18.1
1,2014,2,2Q14,18.7
2,2014,3,3Q14,19.3
3,2014,4,4Q14,19.9
4,2015,1,1Q15,20.5


In [3]:
import pixiedust
display(churnDataRate)


YEAR,QUARTER,QUARTER_YEAR,CHURN_RATE
2014,1,1Q14,18.1
2014,2,2Q14,18.7
2014,3,3Q14,19.3
2014,4,4Q14,19.9
2015,1,1Q15,20.5
2015,2,2Q15,21.1
2015,3,3Q15,21.7
2015,4,4Q15,22.3
2016,1,1Q16,23.0
2016,2,2Q16,23.6


In [4]:
import brunel
%brunel data('churnDataRate') x(QUARTER_YEAR) y(CHURN_RATE) bar tooltip(#all) axes(x:'Time', y:'Churn Rate':grid) sort(YEAR:ascending) :: width=800, height=500
    

<IPython.core.display.Javascript object>

<div style="background-color: #e7f2fa; color: #333">
  <h3 style="background-color: #6ab0de">CUSTOMER PROFILE</h3>
  <p>Load TEST_SUM.csv from IBM Bluemix Object Store.</p>
</div>

In [5]:
import os, pandas as pd

churnData = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/cust_data.csv',header=None)
churnData.columns = ['CUST_ID', 'SEX', 'AGE', 'EDUCATION', 'INVESTMENT', 'INCOME', 'ACTIVITY', 'CHURN', 'YRLY_AMT', 'AVG_DAILY_TX', 'YRLY_TX', 'AVG_TX_AMT', 'NEGTWEETS', 'STATE','EDUCATION_GROUP', 'TWITTERID', 'CHURN_LABEL']
churnData.head()



,CUST_ID,SEX,AGE,EDUCATION,INVESTMENT,INCOME,ACTIVITY,CHURN,YRLY_AMT,AVG_DAILY_TX,YRLY_TX,AVG_TX_AMT,NEGTWEETS,STATE,EDUCATION_GROUP,TWITTERID,CHURN_LABEL
0,1009520370,F,63,4,0.0,13035.0,3,0,6339.0703,0.663014,242,26.0,2,MD,Associate degree,0,False
1,1009520380,F,59,1,0.0,18267.0,0,0,7957.1406,0.654795,239,33.0,2,ME,High school graduate,0,False
2,1009520390,M,31,2,111192.0,129276.0,2,0,31141.1020,0.797260,291,107.0,2,MS,Bachelors degree,0,False
3,1009520400,M,50,1,0.0,17867.0,2,0,7946.0780,0.660274,241,32.0,4,UT,High school graduate,0,False
4,1009520410,F,53,1,0.0,17576.0,1,0,7961.8086,0.663014,242,32.0,4,AK,High school graduate,0,False


In [6]:
groupedByState = churnData.groupby("STATE").agg({"INCOME" : np.average}).rename(columns={"INCOME" : "mean_income"})
groupedByState.head()


,mean_income
STATE,
AK,33184.623529
AL,25630.486842
AR,26104.265823
AZ,25862.160920
CA,73914.220149


## Income by state

In [7]:
import brunel
%brunel data('groupedByState') map key(STATE) x(STATE) color(mean_income) label(STATE) tooltip(#all) :: width=800, height=500
    

<IPython.core.display.Javascript object>

## Distribution by churn

In [8]:
import brunel
%brunel data('churnData') x(AGE) y(#count:linear) color(CHURN_LABEL) bin(AGE) interaction(select) stack bar tooltip(#all) filter(CHURN_LABEL) legends(none) | x(AVG_DAILY_TX) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_DAILY_TX) stack bar tooltip(#all) axes(x:10:'AVG DAILY TX', y)| x(AVG_TX_AMT) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_TX_AMT) stack bar tooltip(#all)  axes(x:10:'AVG_TX_AMT', y) legends(none) | x(INCOME) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(INCOME) stack bar tooltip(#all) tooltip(#all) axes(x:10:'INCOME', y) legends(none):: width=800, height=600


<IPython.core.display.Javascript object>

In [9]:
import brunel
%brunel data('churnData') x(SEX) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(SEX) interaction(select) filter(CHURN_LABEL) axes(x:'GENDER', y) legends(none) | x(EDUCATION_GROUP) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(#count) opacity(#selection)  axes(x:'EDUCATION', y)  | x(ACTIVITY) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(ACTIVITY) opacity(#selection) legends(none) :: width=800, height=600


<IPython.core.display.Javascript object>

In [10]:
churnData.head()

,CUST_ID,SEX,AGE,EDUCATION,INVESTMENT,INCOME,ACTIVITY,CHURN,YRLY_AMT,AVG_DAILY_TX,YRLY_TX,AVG_TX_AMT,NEGTWEETS,STATE,EDUCATION_GROUP,TWITTERID,CHURN_LABEL
0,1009520370,F,63,4,0.0,13035.0,3,0,6339.0703,0.663014,242,26.0,2,MD,Associate degree,0,False
1,1009520380,F,59,1,0.0,18267.0,0,0,7957.1406,0.654795,239,33.0,2,ME,High school graduate,0,False
2,1009520390,M,31,2,111192.0,129276.0,2,0,31141.1020,0.797260,291,107.0,2,MS,Bachelors degree,0,False
3,1009520400,M,50,1,0.0,17867.0,2,0,7946.0780,0.660274,241,32.0,4,UT,High school graduate,0,False
4,1009520410,F,53,1,0.0,17576.0,1,0,7961.8086,0.663014,242,32.0,4,AK,High school graduate,0,False


In [11]:
groupedchurnByState = churnData.groupby("STATE").agg({"CHURN" : np.average}).rename(columns={"CHURN" : "mean_churn"})
groupedchurnByState.head()


,mean_churn
STATE,
AK,0.000000
AL,0.000000
AR,0.000000
AZ,0.000000
CA,0.645522


In [12]:
import brunel
%brunel data('groupedchurnByState') map key(STATE) x(STATE) color(mean_churn) label(STATE) tooltip(#all) :: width=800, height=500


<IPython.core.display.Javascript object>

<p><a href="https://9.30.250.55:31843/#/notebook/jupyter/Bank_Cust_Churn_Analysis/Part%202%20-%20Create%20Churn%20Model">Part 2: Deploy Churn Model</a></p>